<a href="https://colab.research.google.com/github/Manan-Parakh/Road-Inspection-Using-YOLOv5/blob/main/Road_inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install YOLOv5 and dependencies
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

# Install additional dependencies

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install opencv-python numpy matplotlib tqdm


Cloning into 'yolov5'...
remote: Enumerating objects: 17274, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 17274 (delta 1), reused 0 (delta 0), pack-reused 17271 (from 3)
Receiving objects: 100% (17274/17274), 16.13 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (11859/11859), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00

# New Section

In [ ]:
%cd /content
!curl -L https://public.roboflow.com/ds/sidzThkQ0L?key=hCGLlEj4I3 > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   910  100   910    0     0   2602      0 --:--:-- --:--:-- --:--:--  2600
100 46.0M  100 46.0M    0     0  24.9M      0  0:00:01  0:00:01 --:--:-- 51.6M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/img-105_jpg.rf.3fe9dff3d1631e79ecb480ff403bcb86.jpg  
 extracting: test/images/img-107_jpg.rf.2e40485785f6e5e2efec404301b235c2.jpg  
 extracting: test/images/img-146_jpg.rf.61be25b3053a51f622a244980545df2b.jpg  
 extracting: test/images/img-161_jpg.rf.211541e7178a4a93ec0680f26b905427.jpg  
 extracting: test/images/img-168_jpg.rf.af3590e07b06b43e91fa53990ff94af3.jpg  
 extracting: test/images/img-179_jpg.rf.8632eb0d9b75fefe144829e67b75015a.jpg  
 extracting: test/i

In [ ]:
%cat data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['pothole']

In [ ]:
!ls -l /content/


total 32
-rw-r--r-- 1 root root   69 Nov  1  2020 data.yaml
-rw-r--r-- 1 root root  882 Nov  1  2020 README.dataset.txt
-rw-r--r-- 1 root root  377 Nov  1  2020 README.roboflow.txt
drwxr-xr-x 1 root root 4096 Mar 17 13:32 sample_data
drwxr-xr-x 4 root root 4096 Nov  1  2020 test
drwxr-xr-x 4 root root 4096 Nov  1  2020 train
drwxr-xr-x 4 root root 4096 Nov  1  2020 valid
drwxr-xr-x 9 root root 4096 Mar 18 21:59 yolov5


In [ ]:
%cd /content

/content


In [ ]:
import yaml
with open("data.yaml", 'r') as stream:
    num_classes = int(yaml.safe_load(stream)['nc'])

In [ ]:
num_classes

1

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
import torch

In [ ]:
from IPython.display import Image, clear_output

In [ ]:
import numpy as np

In [ ]:
%cd yolov5
from yolov5.models.common import Focus, Conv, BottleneckCSP, SPP, Concat

/content/yolov5


In [ ]:
from yolov5.models.yolo import Detect

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
import torch.nn as nn  # Or simply: import torch.nn

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: num_classes  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
@register_line_cell_magic
def writetemplate(line, cell):
    yaml_content = f'{cell}'  # Use an f-string here
    print("YAML content:")
    print(yaml_content)  # Print the YAML content for review
    with open(line, 'w') as f:
        f.write(yaml_content)

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

Streaming output truncated to the last 5000 lines.
      23/99      2.38G    0.07214    0.03817          0         77        416:  27% 8/30 [00:01<00:03,  7.27it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99      2.38G    0.07274    0.03877          0         94        416:  30% 9/30 [00:01<00:03,  6.51it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99      2.38G    0.07267    0.03848          0         81        416:  33% 10/30 [00:01<00:03,  6.35it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99      2.38G    0.07288    0.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python detect.py --weights runs/train/yolov5s_results/weights/best.pt --img 416 --conf 0.4 --source ../test/images

detect: weights=['runs/train/yolov5s_results/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
image 1/67 /content/test/images/img-105_jpg.rf.3fe9dff3d1631e79ecb480ff403bcb86.jpg: 416x416 1 pothole, 31.5ms
image 2/67 /content/test/images/img-107_jpg.rf.2e40485785f6e5e2efec404301b235c2.jpg: 416x416 1 pothole, 9.2ms
image 3/67 /content/test/images/img-146_jpg.rf.61be25b3053a51f622a244980545df2b.jpg: 416x41

In [ ]:
!cp -r /content/yolov5 /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/

# Convert YOLOv5 to ONNX

In [ ]:
!python /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py \
  --weights /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/runs/train/yolov5s_results/weights/best.pt \
  --include onnx

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 104.0 MB/s eta 0:00:00


In [ ]:
import onnx
model = onnx.load("best.onnx")
print(onnx.helper.printable_graph(model.graph))

In [ ]:
!cat /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/yolov5s.yaml | grep "ch:"

In [ ]:
!ls /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/

common.py	     hub	  segment  yolov5l.yaml  yolov5s.yaml
custom_yolov5s.yaml  __init__.py  tf.py    yolov5m.yaml  yolov5x.yaml
experimental.py      __pycache__  yolo.py  yolov5n.yaml


In [ ]:
!cat /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/yolov5s.yaml


# Ultralytics 🚀 AGPL-3.0 License - https://ultralytics.com/license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, 

In [ ]:
import cv2

img = cv2.imread("/content/train/images/img-100_jpg.rf.233751cac3399aa80b5e58a2afbb753a.jpg")  # Loads image
print("Image shape:", img.shape)  # Should print (H, W, 3)

Image shape: (720, 720, 3)


In [ ]:
!cat /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/yolov5s.yaml | grep "nc:"


nc: 80 # number of classes


In [ ]:
import torch

# Load your trained YOLOv5 model
model_path = "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/runs/train/yolov5s_results/weights/best.pt"
model = torch.load(model_path, map_location="cpu")  # Load on CPU

# Get the first convolutional layer
first_layer = model["model"].model[0]  # Usually the first Conv layer

print("First layer weights shape:", first_layer.conv)

First layer weights shape: Conv(
  (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
  (act): SiLU(inplace=True)
)


## The model expects 12 channels

In [ ]:
import torch
import torch.nn as nn

# Load model
model_path = "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/runs/train/yolov5s_results/weights/best.pt"
model = torch.load(model_path, map_location="cpu")["model"]

# Get the first Conv layer inside the custom 'Conv' module
conv1 = model.model[0].conv  # First layer is a 'Conv' module, inside it is Conv2d

# Create a new Conv2d layer with 3 input channels instead of 12
new_conv1 = nn.Conv2d(
    in_channels=3,
    out_channels=conv1.conv.out_channels,  # Correct way to access out_channels
    kernel_size=conv1.conv.kernel_size,
    stride=conv1.conv.stride,
    padding=conv1.conv.padding,
    bias=False
)

# Copy weights (only take the first 3 channels from the original weights)
with torch.no_grad():
    new_conv1.weight[:] = conv1.conv.weight[:, :3, :, :]

# Replace the old Conv2d layer inside the 'Conv' module
model.model[0].conv = new_conv1

# Save the modified model
torch.save({"model": model}, "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5s_3channel.pt")

print("✅ Successfully converted model to accept 3-channel input!")

✅ Successfully converted model to accept 3-channel input!


In [ ]:
import torch

# Load model
model_path = "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/runs/train/yolov5s_results/weights/best.pt"
model = torch.load(model_path, map_location="cpu")["model"]

# Print the first Conv layer
first_layer = model.model[0]
print(first_layer)  # Should show Conv2d(3, ...), but likely shows Conv2d(12, ...)

Focus(
  (conv): Conv(
    (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
)


In [ ]:
def debug_input_hook(module, input, output):
    print(f"🚨 Input to {module} has shape: {input[0].shape}")

# Attach the hook to the first Conv layer
model.model[0].register_forward_hook(debug_input_hook)

# Run a dummy inference
dummy_input = torch.randn(1, 3, 320, 320).half()  # Convert input to Float16
  # A normal 3-channel image
model(dummy_input)

🚨 Input to Focus(
  (conv): Conv(
    (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
) has shape: torch.Size([1, 3, 320, 320])
🚨 Input to Focus(
  (conv): Conv(
    (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
) has shape: torch.Size([1, 3, 320, 320])


(tensor([[[6.72656e+00, 3.53125e+00, 2.73594e+01, 1.62031e+01, 1.60933e-04, 1.00000e+00],
          [1.23359e+01, 5.39062e+00, 2.32031e+01, 2.09219e+01, 1.52349e-04, 1.00000e+00],
          [2.00000e+01, 3.92188e+00, 2.76250e+01, 2.03750e+01, 6.30021e-05, 1.00000e+00],
          ...,
          [2.51500e+02, 2.92250e+02, 5.24062e+01, 1.19312e+02, 4.65631e-04, 1.00000e+00],
          [2.74500e+02, 2.95500e+02, 3.42500e+01, 1.08000e+02, 3.22580e-04, 1.00000e+00],
          [2.87000e+02, 2.86000e+02, 3.91562e+01, 9.03125e+01, 2.26974e-04, 1.00000e+00]]], dtype=torch.float16),
 [tensor([[[[[ 7.09473e-01, -1.17249e-01,  1.56445e+00,  2.33765e-01, -8.73438e+00,  1.09766e+01],
             [ 8.33130e-02,  3.51318e-01,  1.16211e+00,  5.51270e-01, -8.78906e+00,  1.08906e+01],
             [-6.25849e-05, -1.92566e-02,  1.59375e+00,  5.14160e-01, -9.67188e+00,  1.08906e+01],
             ...,
             [-5.42480e-01, -3.40332e-01,  1.59570e+00,  1.17383e+00, -1.17422e+01,  1.05391e+01],
       

In [ ]:
import torch
import torch.nn as nn

# Load Model
model_path = "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/runs/train/yolov5s_results/weights/best.pt"
model = torch.load(model_path, map_location="cpu")["model"]

# Print the first Focus layer
print(model.model[0])

# Replace first layer's conv to take 3 channels instead of 12
new_conv = nn.Conv2d(
    in_channels=3,  # ✅ Force it to take 3 channels
    out_channels=model.model[0].conv.conv.out_channels,
    kernel_size=model.model[0].conv.conv.kernel_size,
    stride=model.model[0].conv.conv.stride,
    padding=model.model[0].conv.conv.padding,
    bias=False
)

# Copy weights (only first 3 channels)
with torch.no_grad():
    new_conv.weight[:] = model.model[0].conv.conv.weight[:, :3, :, :]

# Replace in model
model.model[0].conv.conv = new_conv

# Save the fixed model
torch.save({"model": model}, "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt")

print("✅ Model fixed and saved!")

Focus(
  (conv): Conv(
    (conv): Conv2d(12, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
)
✅ Model fixed and saved!


In [ ]:
!python /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py \
  --weights /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt \
  --include onnx

export: data=../../../../data/coco128.yaml, weights=['/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7243926 parameters, 0 gradients
Traceback (most recent call last):
  File "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py", line 1546, in <module>
    main(opt)
  File "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py", line 1541, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    retur

In [ ]:
import torch
import torch.nn as nn

# Load model
model = torch.load(model_path, map_location="cpu")["model"]

# Get the first Focus layer
focus_layer = model.model[0]

# Fix the first Conv layer to accept 3 channels instead of 12
fixed_conv = nn.Conv2d(
    in_channels=3,  # ✅ Fix input channels to 3
    out_channels=focus_layer.conv.conv.out_channels,
    kernel_size=focus_layer.conv.conv.kernel_size,
    stride=focus_layer.conv.conv.stride,
    padding=focus_layer.conv.conv.padding,
    bias=False
)

# Copy weights (use only the first 3 channels)
with torch.no_grad():
    fixed_conv.weight[:] = focus_layer.conv.conv.weight[:, :3, :, :]

# Replace the old Conv layer with the fixed one
model.model[0].conv.conv = fixed_conv

# Save the corrected model
torch.save({"model": model}, "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt")
print("✅ Model fixed and saved!")

✅ Model fixed and saved!


In [ ]:
!python /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py \
  --weights /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt \
  --include onnx

export: data=../../../../data/coco128.yaml, weights=['/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/fixed_yolov5.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7243926 parameters, 0 gradients
Traceback (most recent call last):
  File "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py", line 1546, in <module>
    main(opt)
  File "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/export.py", line 1541, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    retur

In [ ]:
import fileinput

yaml_path = "/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/custom_yolov5s.yaml"

# Read and modify the file
with fileinput.FileInput(yaml_path, inplace=True) as file:
    for line in file:
        # Replace incorrect Focus layer definition
        print(line.replace("Focus, [64, 3]", "Focus, [64]"), end="")

print("✅ YAML file updated successfully!")


✅ YAML file updated successfully!


In [ ]:
!cat /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/custom_yolov5s.yaml | grep "Focus"


  [[-1, 1, Focus, [64]],  # 0-P1/2


In [ ]:
# Retrain
%cd /content/
!python train.py --img 416 --batch 16 --epochs 100 --data 'data.yaml' --cfg ./models/custom_yolov5s.yaml --weights '' --name yolov5s_results

/content
python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 100 --data '../data.yaml' --cfg /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/yolov5/models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

Streaming output truncated to the last 5000 lines.
      23/99       2.4G    0.07222    0.03703          0         77        416:  27% 8/30 [00:01<00:02,  7.82it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99       2.4G     0.0725    0.03771          0         94        416:  30% 9/30 [00:01<00:02,  7.66it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99       2.4G    0.07249    0.03743          0         81        416:  33% 10/30 [00:01<00:02,  6.96it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      23/99       2.4G    0.07257    0.

In [ ]:
!cp /content/yolov5/runs/train/yolov5s_results/weights/best.pt /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/

In [ ]:
!python /content/yolov5/export.py --weights /content/drive/MyDrive/Road-Inspection-Using-YOLOv5/best.pt --include onnx

export: data=data/coco128.yaml, weights=['/content/drive/MyDrive/Road-Inspection-Using-YOLOv5/best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, cache=, simplify=False, mlmodel=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-399-g8cc44963 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
Traceback (most recent call last):
  File "/content/yolov5/export.py", line 1546, in <module>
    main(opt)
  File "/content/yolov5/export.py", line 1541, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/content/yolov5/export.py", line 1407, in